In [2]:
import sys,os, traceback
import cx_Oracle
import datetime, time
import pandas as pd
import logging
BASE_DIR = r"C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test"
# BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.append(BASE_DIR)

from core import configadmin,log_ctrl,color_me

config_path = os.path.join(BASE_DIR, 'conf')
config_name = 'setting.ini'
d_set = configadmin.ConfigAdmin(config_path, config_name)
loggs = log_ctrl.de8ug_log(logger_name='camp_res', log_file=os.path.join(BASE_DIR, 'log', f'{time.strftime("%Y%m",time.localtime())}-debug.log'), level = logging.DEBUG)

In [3]:
class campaign_res:
    def __init__(self, campaign_cd, communication_cd, url_check, launch_time, End_time=None, model='t', group=None):
        """
        计算Read Rate (排除Remove，Control Group)
        campaign_cd:当campaign_cd不为空("XNA")且communication_cd为空("XNA")时，计算该campaign下，launch_time之后所有communication阅读率(不按人消重计算)。
        communication_cd:当Communication_cd不为空时，优先按Communication_cd计算Read Rate(按人消重计算).当Campaign_cd & Communication_cd 同时不为空时(不按人消重计算)
        url_check:是否检查连接点击率(SMT, Wechat适用)
        launch_time:检查首次Campaign起始时间
        End_time:检查第n次Campagin起始时间(默认为空)
        model:
            -t —— Till Now(默认)
            -l —— By Launch Time
            -d —— By day
            -g —— By Group:适用该分组功能时，communication_cd不能为空，且数量必须与group一致，中间适用空格分隔
        """
        self.campaign_cd = campaign_cd if len(campaign_cd) > 3 else None
        self.communication_cd = str(communication_cd).replace(' ',"','") if len(communication_cd) > 4 else None
        self.url_check = url_check
        self.launch_time = launch_time
        self.End_time = End_time
        self.model = model
        self.user = d_set.read_config("SAS_db", "user")
        self.password = d_set.read_config("SAS_db", "password")
        self.service = d_set.read_config("SAS_db", "service")
        if model == 'g': 
            if group != None and self.communication_cd != None:
                self.group = {'COMMUNICATION_CD':communication_cd.split(" "),'GROUP_TYPE':group.split(" ")}
            else:
                loggs.error('Parameter error: Check your group condition!!!')
                raise ValueException("Parameter error: Check your group condition!!!")

    def get_res(self):
        try:
            self.db = cx_Oracle.connect(self.user, self.password, self.service)
            self.con = self.db.cursor()
            if self.communication_cd:
                self.con1 = f"GE.COMMUNICATION_CD IN ('{self.communication_cd}')"
                if self.campaign_cd:
                    self.con1 = self.con1 + f" AND GE.CAMPAIGN_CD = '{self.campaign_cd}'"
            else:
                self.con1 = f"GE.CAMPAIGN_CD = '{self.campaign_cd}'"
                                                                           
            if self.End_time:
                self.con2 = f"GE.PROCESSED_DTTM > TO_DATE('{self.launch_time}', 'yyyy-mm-dd hh24:mi:ss') AND GE.PROCESSED_DTTM < TO_DATE('{self.End_time}', 'yyyy-mm-dd hh24:mi:ss')"
            else:
                self.con2 = f"GE.PROCESSED_DTTM > TO_DATE('{self.launch_time}', 'yyyy-mm-dd hh24:mi:ss')"
            
            oo = f"SELECT GE.RESPONSE_TRACKING_CD,\
                GE.CAMPAIGN_CD,\
                GE.CAMPAIGN_NM,\
                GE.COMMUNICATION_CD,\
                GE.COMMUNICATION_NM,\
                GE.COMMUNICATION_OCCURRENCE_NO,\
                GE.CHANNEL_CD,\
                GE.SUBJECT_TYPE_NM,\
                TRUNC(GE.PROCESSED_DTTM) AS PROCESSED_DTTM\
                FROM APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
                WHERE {self.con1}\
                AND {self.con2}\
                AND GE.CONTROL_GROUP_TYPE_CD IS NULL\
                AND GE.CHANNEL_CD != '_MC'\
                ORDER BY GE.PROCESSED_DTTM"
            
            col = ['RESPONSE_TRACKING_CD','CAMPAIGN_CD','CAMPAIGN_NM','COMMUNICATION_CD','COMMUNICATION_NM','COMMUNICATION_OCCURRENCE_NO','CHANNEL_CD','SUBJECT_TYPE_NM','PROCESSED_DTTM']
            rows = self._excute_sql(oo)

            if len(rows) < 1:
                print(oo)
                logging.info('None Results')
                return False
            
            df1 = pd.DataFrame(data=rows, index=[i[0] for i in rows], columns=col)
            self._a = "="
            if self.model == 't':
                df = df1.drop_duplicates(subset=['CAMPAIGN_CD','CHANNEL_CD'], keep='first')
                self._a = ">="
                self._b = self.con1
            elif self.model == 'l' or self.model == 'd':
                df = df1.sort_index()
            elif self.model == 'g':
                if len(self.group["COMMUNICATION_CD"]) == len(self.group["GROUP_TYPE"]):
                    df2 = pd.DataFrame(data=self.group, index=self.group["COMMUNICATION_CD"], columns=['GROUP_TYPE'])
                    df1 = df1.set_index([df1.COMMUNICATION_CD])
                    df = pd.concat([df1, df2], axis=1, join='inner')
                else:
                    loggs.error('Group Data not matched!')
                    return False
            else:
                loggs.error('Parameter error: No such group type!')
                return False
                                                                           
            num = 0
            dic = {}
            for i in ['SMT', 'GEN', "SMS", 'WCH', 'RPT']:
                if len(df)==num:
                    break
                else:
                    num += len(df.loc[df.CHANNEL_CD==i])
                    dic[i] = self.channel_response(df.loc[df.CHANNEL_CD==i], i)
                                                                           
            self.db.close()
            # loggs.info(dic)
            return dic
        except Exception as e:
            loggs.error(traceback.format_exc())

    def notify_type(self, COMMUNICATION_CD, CHANNEL_CD, OCCURRENCE_NO, PROCESSED_DTTM):
        if type(COMMUNICATION_CD) != str:
            COMMUNICATION_CD = COMMUNICATION_CD.iloc[0]
                                                                           
        if self.model != 't':
            self._b = f"GE.COMMUNICATION_CD = '{COMMUNICATION_CD}'"
                                                                           
        if CHANNEL_CD == 'SMT':
            sql = f"select distinct case when tem.is_popup > 0 then 'POP_UP'  when  tem.is_notify > 0 then 'PUSH' end as Notify_type \
            from app_campaign_cdm.ci_cell_package ge \
            left join OWNER_OUT.OT_SMT_MESSAGE_DATA TA \
            on ta.campaign_track_id = ge.response_tracking_cd \
            left join dm_campaign.v_smt_template tem \
            on ta.template_id = tem.id \
            WHERE {self._b} AND ge.COMMUNICATION_OCCURRENCE_NO {self._a} {OCCURRENCE_NO}\
            AND {self.con2} AND ge.control_group_type_cd is null\
            AND ge.channel_cd != '_MC'"
        res = self._excute_sql(sql)[0][0]
        if res:
            return res
        else:
            print(sql)
            return None

    def channel_response(self, df, channel):
        if channel == "SMT":
            data = df.copy()
            data['Notify_Type'] = df.apply(lambda x: self.notify_type(x['COMMUNICATION_CD'], x['CHANNEL_CD'], x['COMMUNICATION_OCCURRENCE_NO'], x['PROCESSED_DTTM']), axis=1)
            data['Total_Send'] = df.apply(lambda x: self.owner_cnt(x['COMMUNICATION_CD'], x['CHANNEL_CD'], x['COMMUNICATION_OCCURRENCE_NO'], x['PROCESSED_DTTM']), axis=1)
            data['Send_Succ'] = df.apply(lambda x: self.contact_cnt(x['COMMUNICATION_CD'], x['SUBJECT_TYPE_NM'], x['COMMUNICATION_OCCURRENCE_NO'], x['PROCESSED_DTTM']), axis=1)
            data['Delivery_Rate'] = data.apply(lambda x: x['Send_Succ']/x['Total_Send'], axis=1)
            data['Inbox_Read'] = df.apply(lambda x: self.inbox_cnt(x['COMMUNICATION_CD'], x['CHANNEL_CD'], x['SUBJECT_TYPE_NM'], x['COMMUNICATION_OCCURRENCE_NO'], x['PROCESSED_DTTM']), axis=1)
            data['Inbox_Rate'] = data.apply(lambda x: x['Inbox_Read']/x['Send_Succ'], axis=1)
            if len(data.loc[data.Notify_Type=='POP_UP']) > 0:
                data['Popup_Read'] = data.loc[data.Notify_Type=='POP_UP'].apply(lambda x: self.popup_cnt(x['COMMUNICATION_CD'], x['CHANNEL_CD'], x['SUBJECT_TYPE_NM'], x['COMMUNICATION_OCCURRENCE_NO'], x['PROCESSED_DTTM']), axis=1)
                data['Popup_Rate'] = data.apply(lambda x: x['Popup_Read']/x['Send_Succ'], axis=1)
            if len(data.loc[data.Notify_Type=='PUSH']) > 0:
                data['Push_Read'] = data.loc[data.Notify_Type=='PUSH'].apply(lambda x: self.popup_cnt(x['COMMUNICATION_CD'], x['CHANNEL_CD'], x['SUBJECT_TYPE_NM'], x['COMMUNICATION_OCCURRENCE_NO'], x['PROCESSED_DTTM']), axis=1)
                data['Push_Rate'] = data.apply(lambda x: x['Push_Read']/x['Send_Succ'], axis=1)
            if self.url_check == 'a':
                data["Url_Click"] = df.apply(lambda x: self.url_click(x['COMMUNICATION_CD'], x['CHANNEL_CD'], x['SUBJECT_TYPE_NM'], x['COMMUNICATION_OCCURRENCE_NO'], x['PROCESSED_DTTM']), axis=1)
                data['Click_Rate'] = data.apply(lambda x: x['Url_Click']/x['Send_Succ'], axis=1)
            return data
    
    def owner_cnt(self, COMMUNICATION_CD, CHANNEL_CD, OCCURRENCE_NO, PROCESSED_DTTM):
        if type(COMMUNICATION_CD) != str:
            COMMUNICATION_CD = COMMUNICATION_CD.iloc[0]
        
        if CHANNEL_CD == "SMT":
            oo_tb = "OT_SMT_MESSAGE_DATA"
            remove_tb = "LEFT JOIN OWNER_OUT.OT_SMT_FORCED_DELETION RE ON RE.CAMPAIGN_TRACKID_FD = TA.CAMPAIGN_TRACK_ID AND RE.ID_ENTITY_EMPLOYEE = TA.ID_ENTITY_EMPLOYEE"
            remove_con = "AND RE.ID_ENTITY_EMPLOYEE IS NULL"
        
        if self.model != 't':
            self._b = f"GE.COMMUNICATION_CD = '{COMMUNICATION_CD}'"
        
        sql = f"SELECT COUNT(TA.ID_ENTITY_EMPLOYEE) AS OWNER_OUT\
            FROM OWNER_OUT.{oo_tb} TA\
            LEFT JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
            ON GE.RESPONSE_TRACKING_CD = TA.CAMPAIGN_TRACK_ID\
            {remove_tb}\
            WHERE {self._b}\
            and GE.Communication_Occurrence_No {self._a} {OCCURRENCE_NO}\
            and ge.control_group_type_cd is null \
            AND ge.deleted_flg = 'N' \
            AND ge.channel_cd != '_MC'\
            AND TA.ACTIVE_START >= TO_DATE('{PROCESSED_DTTM}','yyyy-mm-dd hh24:mi:ss')\
            {remove_con}"
        res = self._excute_sql(sql)[0][0]
        if res:
            return res
        else:
            print(sql)
            return None

    def contact_cnt(self, COMMUNICATION_CD, SUBJECT_TYPE_NM, OCCURRENCE_NO, PROCESSED_DTTM):
        if type(COMMUNICATION_CD) != str:
            COMMUNICATION_CD = COMMUNICATION_CD.iloc[0]
        
        a = lambda x: "TIP" if x == "Tipper" else "EMP"
        if self.model != 't':
            self._b = f"GE.COMMUNICATION_CD = '{COMMUNICATION_CD}'"
        
        sql = f"SELECT COUNT(HMP.ID_XRM_EMPLOYEE) AS DELIVERY_SUC\
            FROM APP_CAMPAIGN_CDM.CI_CONTACT_HISTORY_{a(SUBJECT_TYPE_NM)} HMP\
            JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
            ON GE.CELL_PACKAGE_SK = HMP.CELL_PACKAGE_SK\
            WHERE {self._b}\
            AND HMP.CONTACT_HISTORY_STATUS_CD <> '_30'\
            AND HMP.CONTACT_HISTORY_STATUS_CD <> '_31'\
            AND ge.COMMUNICATION_OCCURRENCE_NO {self._a} {OCCURRENCE_NO}\
            and ge.control_group_type_cd is null \
            AND ge.deleted_flg = 'N' \
            AND ge.channel_cd != '_MC'\
            AND GE.PROCESSED_DTTM >= TO_DATE('{PROCESSED_DTTM}', 'yyyy-mm-dd hh24:mi:ss')"
        res = self._excute_sql(sql)[0][0]
        if res:
            return res
        else:
            print(sql)
            return None

    def inbox_cnt(self, COMMUNICATION_CD, CHANNEL_CD, SUBJECT_TYPE_NM, OCCURRENCE_NO, PROCESSED_DTTM):
        if type(COMMUNICATION_CD) != str:
            COMMUNICATION_CD = COMMUNICATION_CD.iloc[0]
        
        a = lambda x: "TIP" if x == "Tipper" else "EMP"
        
        if self.model != 't':
            self._b = f"GE.COMMUNICATION_CD = '{COMMUNICATION_CD}'"
                                                                           
        sql = f"SELECT COUNT(DIstinct hmp.id_xrm_employee) as POPUP_READ\
            FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_{a(SUBJECT_TYPE_NM)} HMP\
            JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
            ON GE.CELL_PACKAGE_SK = HMP.CELL_PACKAGE_SK\
            WHERE {self._b}\
            AND HMP.RESPONSE_SK IN ('1029', '1030')\
            AND ge.communication_occurrence_no {self._a} {OCCURRENCE_NO}\
            and ge.control_group_type_cd is null\
            AND ge.deleted_flg = 'N' \
            AND ge.channel_cd != '_MC'\
            AND {self.con2}\
            AND HMP.ID_XRM_EMPLOYEE NOT IN ({self._remove_tb(CHANNEL_CD)}\
            LEFT JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE \
            ON GE.RESPONSE_TRACKING_CD = A.CAMPAIGN_TRACKID_FD \
            WHERE {self._b} AND ge.COMMUNICATION_OCCURRENCE_NO {self._a} {OCCURRENCE_NO}\
            AND {self.con2}\
            AND GE.CONTROL_GROUP_TYPE_CD IS NULL)"
        res = self._excute_sql(sql)[0][0]
        if res:
            return res
        else:
            print(sql)
            return None
                                                                           
    def popup_cnt(self, COMMUNICATION_CD, CHANNEL_CD, SUBJECT_TYPE_NM, OCCURRENCE_NO, PROCESSED_DTTM):
        if type(COMMUNICATION_CD) != str:
            COMMUNICATION_CD = COMMUNICATION_CD.iloc[0]
        
        a = lambda x: "TIP" if x == "Tipper" else "EMP"
        
        if self.model != 't':
            self._b = f"GE.COMMUNICATION_CD = '{COMMUNICATION_CD}'"
                                                                           
        sql = f"SELECT COUNT(distinct hmp.id_xrm_employee) as POPUP_READ\
            FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_{a(SUBJECT_TYPE_NM)} HMP\
            JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
            ON GE.CELL_PACKAGE_SK = HMP.CELL_PACKAGE_SK\
            WHERE {self._b}\
            AND HMP.RESPONSE_SK = '1028'\
            AND ge.communication_occurrence_no {self._a} {OCCURRENCE_NO}\
            and ge.control_group_type_cd is null\
            AND ge.deleted_flg = 'N' \
            AND ge.channel_cd != '_MC'\
            AND {self.con2}\
            AND HMP.ID_XRM_EMPLOYEE NOT IN ({self._remove_tb(CHANNEL_CD)}\
            LEFT JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE \
            ON GE.RESPONSE_TRACKING_CD = A.CAMPAIGN_TRACKID_FD \
            WHERE {self._b} AND ge.COMMUNICATION_OCCURRENCE_NO {self._a} {OCCURRENCE_NO}\
            AND {self.con2}\
            AND GE.CONTROL_GROUP_TYPE_CD IS NULL)"
        
        return self._excute_sql(sql)[0][0]

    def push_cnt(self, COMMUNICATION_CD, CHANNEL_CD, SUBJECT_TYPE_NM, OCCURRENCE_NO, PROCESSED_DTTM):
        if type(COMMUNICATION_CD) != str:
            COMMUNICATION_CD = COMMUNICATION_CD.iloc[0]
        
        a = lambda x: "TIP" if x == "Tipper" else "EMP"
        
        if self.model != 't':
            self._b = f"GE.COMMUNICATION_CD = '{COMMUNICATION_CD}'"
                                                                           
        sql = f"SELECT COUNT(distinct hmp.id_xrm_employee) as POPUP_READ\
            FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_{a(SUBJECT_TYPE_NM)} HMP\
            JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
            ON GE.CELL_PACKAGE_SK = HMP.CELL_PACKAGE_SK\
            WHERE {self._b}\
            AND HMP.RESPONSE_SK = '1031'\
            AND ge.communication_occurrence_no {self._a} {OCCURRENCE_NO}\
            and ge.control_group_type_cd is null\
            AND ge.deleted_flg = 'N' \
            AND ge.channel_cd != '_MC'\
            AND {self.con2}\
            AND HMP.ID_XRM_EMPLOYEE NOT IN ({self._remove_tb(CHANNEL_CD)}\
            LEFT JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE \
            ON GE.RESPONSE_TRACKING_CD = A.CAMPAIGN_TRACKID_FD \
            WHERE {self._b} AND ge.COMMUNICATION_OCCURRENCE_NO {self._a} {OCCURRENCE_NO}\
            AND {self.con2}\
            AND GE.CONTROL_GROUP_TYPE_CD IS NULL)"
        res = self._excute_sql(sql)[0][0]
        if res:
            return res
        else:
            print(sql)
            return None

    def url_click(self, COMMUNICATION_CD, CHANNEL_CD, SUBJECT_TYPE_NM, OCCURRENCE_NO, PROCESSED_DTTM):
        if type(COMMUNICATION_CD) != str:
            COMMUNICATION_CD = COMMUNICATION_CD.iloc[0]
        
        a = lambda x: "TIP" if x == "Tipper" else "EMP"
        
        if self.model != 't':
            self._b = f"GE.COMMUNICATION_CD = '{COMMUNICATION_CD}'"
                                                                           
        sql = f"SELECT COUNT(distinct HMP.Id_Xrm_Employee) AS URL_CLICK\
            FROM APP_CAMPAIGN_CDM.CI_RESPONSE_HISTORY_{a(SUBJECT_TYPE_NM)} HMP\
            LEFT JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE\
            ON HMP.CELL_PACKAGE_SK = GE.CELL_PACKAGE_SK\
            WHERE {self._b} AND {self.con2}\
            AND ge.communication_occurrence_no {self._a} {OCCURRENCE_NO}\
            AND HMP.External_Response_Info_Id2 is not null\
            and ge.control_group_type_cd is null\
            AND ge.deleted_flg = 'N' \
            AND ge.channel_cd != '_MC'\
            AND HMP.ID_XRM_EMPLOYEE NOT IN ({self._remove_tb(CHANNEL_CD)}\
            LEFT JOIN APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE \
            ON GE.RESPONSE_TRACKING_CD = A.CAMPAIGN_TRACKID_FD \
            WHERE {self._b} AND ge.COMMUNICATION_OCCURRENCE_NO {self._a} {OCCURRENCE_NO}\
            AND {self.con2}\
            AND GE.CONTROL_GROUP_TYPE_CD IS NULL)\
            ORDER BY GE.COMMUNICATION_CD"
        res = self._excute_sql(sql)[0][0]
        if res:
            return res
        else:
            print(sql)
            return None
                                                                           
    def _remove_tb(self, channel_cd):
        if channel_cd == "SMT":
            remove_con_his ="SELECT A.ID_ENTITY_EMPLOYEE_TYPE || '.' || A.ID_ENTITY_EMPLOYEE \
                FROM OWNER_OUT.OT_SMT_FORCED_DELETION A"
            return remove_con_his
        else:
            return False
    
    def _excute_sql(self, sql):
        try:
            self.con.execute(sql)
            rows = self.con.fetchall()
            return rows
        except Exception as e:
            print(e)
            print(sql)

In [4]:
campaign_cd = 'XNA'
COMMUNICATION_CD = 'COMM9949'
LAUNCH_TIME = '2020/7/9 13:42:16'
END_TIME = '2020/7/9 13:42:16'
URL_CHICK = 'n'
MODEL = 't'
GROUP = None
test = campaign_res(campaign_cd, COMMUNICATION_CD, URL_CHICK, LAUNCH_TIME, END_TIME, MODEL, GROUP)

In [5]:
data = test.get_res()

SELECT GE.RESPONSE_TRACKING_CD,                GE.CAMPAIGN_CD,                GE.CAMPAIGN_NM,                GE.COMMUNICATION_CD,                GE.COMMUNICATION_NM,                GE.COMMUNICATION_OCCURRENCE_NO,                GE.CHANNEL_CD,                GE.SUBJECT_TYPE_NM,                TRUNC(GE.PROCESSED_DTTM) AS PROCESSED_DTTM                FROM APP_CAMPAIGN_CDM.CI_CELL_PACKAGE GE                WHERE GE.COMMUNICATION_CD IN ('COMM9949')                AND GE.PROCESSED_DTTM > TO_DATE('2020/7/9 13:42:16', 'yyyy-mm-dd hh24:mi:ss') AND GE.PROCESSED_DTTM < TO_DATE('2020/7/9 13:42:16', 'yyyy-mm-dd hh24:mi:ss')                AND GE.CONTROL_GROUP_TYPE_CD IS NULL                AND GE.CHANNEL_CD != '_MC'                ORDER BY GE.PROCESSED_DTTM
